In [ ]:
# 1. Обработка датасета

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
train_df = pd.read_csv("/kaggle/input/forest-cover-type-prediction/train.csv")
test_df  = pd.read_csv("/kaggle/input/forest-cover-type-prediction/test.csv")

,Id,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,...,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type
0,1,2596,51,3,258,0,510,221,232,148,...,0,0,0,0,0,0,0,0,0,5
1,2,2590,56,2,212,-6,390,220,235,151,...,0,0,0,0,0,0,0,0,0,5
2,3,2804,139,9,268,65,3180,234,238,135,...,0,0,0,0,0,0,0,0,0,2
3,4,2785,155,18,242,118,3090,238,238,122,...,0,0,0,0,0,0,0,0,0,2
4,5,2595,45,2,153,-1,391,220,234,150,...,0,0,0,0,0,0,0,0,0,5


In [3]:
print("Train shape:", train_df.shape)
print("Test shape :", test_df.shape)

Train shape: (15120, 56)
Test shape : (565892, 55)


In [4]:
target = train_df["Cover_Type"]
train_df = train_df.drop(['Cover_Type'], axis=1)

In [7]:
def data_processing(df):
    df["Hillshade_Total"] = df["Hillshade_9am"] + df["Hillshade_Noon"] + df["Hillshade_3pm"]
    df["Hillshade_Mean"] = df["Hillshade_Total"] / 3.0
    df = df.drop(['Hillshade_9am'], axis=1)
    df = df.drop(['Hillshade_Noon'], axis=1)
    df = df.drop(['Hillshade_3pm'], axis=1)
    df = df.drop(['Hillshade_Total'], axis=1)
    df = df.drop(['Id'], axis=1)
    return df
    
test_df_ = test_df.copy()
train_df = data_processing(train_df)
test_df = data_processing(test_df)

In [9]:
train_df['Soil_Type15'].value_counts()

Soil_Type15
0    15120
Name: count, dtype: int64

In [ ]:
# 2. Выбор модели

In [10]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
target = le.fit_transform(target)

X_train, X_valid, y_train, y_valid = train_test_split(
    train_df, target, test_size=0.2, random_state=42, stratify=target
)

In [11]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier


classifiers = {
    'XGB': {
        'model': XGBClassifier(random_state=42, eval_metric="mlogloss"),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [4, 6, 8],
            'learning_rate': [0.01, 0.1, 0.3],
            'subsample': [0.8, 1.0]
        }
    }
}

In [ ]:
from sklearn.model_selection import GridSearchCV

results = {}

for name, clf_info in classifiers.items():
    grid_search = GridSearchCV(
        estimator=clf_info['model'],
        param_grid=clf_info['params'],
        cv=3,
        scoring='accuracy',
        n_jobs=-1,
        verbose=1
    )

    grid_search.fit(train_df, target)

    best_model = grid_search.best_estimator_
    best_score = grid_search.best_score_
    results[name] = {
        'model': best_model,
        'best_params': grid_search.best_params_,
        'best_accuracy': best_score
    }



best_classifier_name = max(results.keys(), key=lambda k: results[k]['best_accuracy'])
best_model = results[best_classifier_name]['model']

y_pred_test = best_model.predict(test_df)
y_pred_test = le.inverse_transform(y_pred_test)

submission = pd.DataFrame({
    'Id': test_df_['Id'],
    'Cover_Type': y_pred_test
})

submission.to_csv('submission.csv', index=False)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


{'XGB': {'model': XGBClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bynode=None,
                colsample_bytree=None, device=None, early_stopping_rounds=None,
                enable_categorical=False, eval_metric='mlogloss',
                feature_types=None, gamma=None, grow_policy=None,
                importance_type=None, interaction_constraints=None,
                learning_rate=0.1, max_bin=None, max_cat_threshold=None,
                max_cat_to_onehot=None, max_delta_step=None, max_depth=8,
                max_leaves=None, min_child_weight=None, missing=nan,
                monotone_constraints=None, multi_strategy=None, n_estimators=300,
                n_jobs=None, num_parallel_tree=None, objective='multi:softprob', ...),
  'best_params': {'learning_rate': 0.1,
   'max_depth': 8,
   'n_estimators': 300,
   'subsample': 0.8},
  'best_accuracy': 0.7853835978835978}}